In [ ]:
# This code block imports essential libraries. No changes are required here 
# for new users unless additional functionality is needed.
import cv2
import numpy as np
import os
from glob import glob
import pandas as pd
import trimesh
import pandas as pd
import os
from glob import glob
import re

In [ ]:
# Read the Excel sheet into a DataFrame

# Specifying the path to the Excel file. This file is expected to be created by a separate script named '1_cropping_desired_crystal_coordinates.ipynb'.
excel_path = 'D:/Radium/Results_auto/Input_Parameters.xlsx' # make sure that the input folder is for Orthorhombic Bipyramidal crstals

# Loading a 3D mesh from an STL file. This STL file represents a Orthorhombic Bipyramidal crystal structure.
your_mesh = trimesh.load_mesh('D:/Radium/Raman_3D/Crystal_type_5/crystal5.stl')

In [ ]:
# Specify the Molar Volume value to be used later in the calculation later in the script
# Molar volume (cm^3 mol^-1)
molar_volume_cm3 = 53.7

This code block defines two functions for specific tasks:

    Finding Input Folders: The find_xy_folders function is designed to locate folders within a specified root directory that match a naming pattern based on an 'XY number'. It uses the glob module to search through directories recursively. This function is useful for organizing or categorizing files and folders based on naming conventions. This function is compatible with the current processing method that destiguishes different captured locations in the microfluidic chip and identify each region with a distinct (XY) number. If this process method is changed, then this function should be changed accordingly.

    Calculating Tetrahedron Volume: The tetrahedron_volume function calculates the volume of a tetrahedron (a four-sided 3D shape) given the coordinates of its four vertices. This calculation is performed using vector operations (cross and dot products) provided by NumPy, illustrating the function's application in geometrical or 3D mesh analysis.

In [ ]:
# Function to find input folders

# Define a function that finds folders based on a given XY number pattern within a specified root path.
def find_xy_folders(root_path, xy_number):
    # Forming a search pattern using the root_path and xy_number. The '**' allows for searching in all subdirectories.
    search_pattern = os.path.join(root_path, '**', f'XY{xy_number}')
    # Uncomment the following line to print the search pattern for debugging.
    # print(search_pattern)

    # Using glob to find all folders that match the search pattern. The 'recursive=True' parameter allows searching in all subdirectories.
    input_folders = glob(search_pattern, recursive=True)
    # Uncomment the following line to print the list of found folders for debugging.
    # print(input_folders)

    # Returning the list of folders found.
    return input_folders

# Function to calculate the volume of a tetrahedron

# Define a function to calculate the volume of a tetrahedron given its four vertices (a, b, c, d).
def tetrahedron_volume(a, b, c, d):
    # Calculating the volume using a formula based on the cross product and dot product.
    return np.abs(np.dot(a-d, np.cross(b-d, c-d))) / 6.0


This code block performs complex image processing to identify and analyze rhomboid shapes in images. It integrates these results with 3D mesh volume calculations and consolidates data from various sources into cohesive DataFrames. The results are saved and summarized, providing a comprehensive output of the analysis.
Usage Tips:

   - Ensure that all prerequisite files (images, CSV, Excel) are correctly placed in the specified directories.
   - Adjust parameters (like contour area thresholds, scaling factors) according to the specifics of your dataset and analysis requirements.
   - Review the outputs (saved images, CSV files) regularly to ensure that the processing is happening as expected.

Note on Adjusting Parameters:

   - Bilateral Filter: The parameters 9, 75, 75 in cv2.bilateralFilter can be adjusted. The first one is the diameter of the pixel neighborhood, the second is the filter sigma in the color space, and the third is the filter sigma in the coordinate space. Adjust these for different levels of noise reduction and edge preservation.
   - Adaptive Thresholding: The 15 and 3 in cv2.adaptiveThreshold are the block size and a constant subtracted from the mean, respectively. Vary these to change the binarization effect based on local image characteristics.

Explanation of Parameters and Processes:

   - Contour Finding: The cv2.findContours function detects contours in an image. cv2.RETR_EXTERNAL retrieves only the external contours, and cv2.CHAIN_APPROX_NONE stores the full contour without compression.

   - Contour Area: cv2.contourArea calculates the area of each contour. Contours smaller than min_contour_area (50 pixels in this case) are ignored.

   - Nested Contours: The code checks for contours nested within other contours to filter them out. This is done by comparing the bounding rectangles of each contour pair.

   - Rhomboid Detection: The script approximates each contour to a polygon and checks if it has four sides and angles within a specified threshold (angle_threshold), which is set to 25 degrees. Adjust this value to be more or less strict about the shape's angular accuracy.
   - Skipping Images: If skip_image is True, the original image is saved without further processing, and a row with empty analysis results is added to the DataFrame.

   - Rhomboid Contour Processing: If multiple rhomboid contours are found, the code retains only the largest one based on the contour area.

   - Drawing and Analyzing Rhomboids: The script draws contours around the detected rhomboids and calculates their diagonal lengths and area. The area is then converted from pixels to micrometers squared using the conversion_rate.
   - 3D Mesh Volume Calculation: The code calculates the volume of a 3D mesh by summing the volumes of tetrahedra formed by the mesh faces and the origin. It then scales the mesh based on the rhomboid dimensions and recalculates the volume and surface area.

In [ ]:
# # Read the Excel sheet into a DataFrame
df = pd.read_excel(excel_path, dtype={'xy_number': str})  # Read xy_number as string
main_df = pd.DataFrame()

# New DataFrame for crystal size
crystal_size_df = pd.DataFrame(columns=['exp_number', 'folder_number', 'diagonal_1'])

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Initialize a variable to store the previous area value
    previous_area = 0.0  
    # Extracting values from the row
    exp_number = row['exp_number']
    root_path = row['root_path']
    xy_number = row['xy_number']  # This will be a string
    folder_name = row['folder_name']
    folder_number = int(re.search(r'\d+', folder_name).group())
    coord_str = row['coord_str']
    
    # Use your function to find input folders
    input_folders = find_xy_folders(root_path, xy_number)
    
#     # Extract coordinates from coord_str
    match = re.search(r"\[(\d+):(\d+), (\d+):(\d+)\]", coord_str)
    if match:
        x_start, x_end, y_start, y_end = map(int, match.groups())

    found_crystals = 0

    # Construct the output folder path
    output_folder = f"D:/Radium/Results_auto/{exp_number}/{folder_name}/"
    conversion_rate = 0.18  # micrometers/pixel
    output_csv_filename = f"{exp_number}_Orthorhombic_Bipyramidal.csv"
    
    # Creating the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through each folder in the list
    image_paths = []
    for input_folder in input_folders:
        image_paths.extend(glob(os.path.join(input_folder, "*.tif")))
        
    # Creating a DataFrame for storing image analysis results
    df = pd.DataFrame(columns=["ImageName", "RectangularArea_micrometer^2", 
                               "Volume_micrometer^3", "SurfaceArea_micrometer^2",
                               "Number_of_Moles"])
    
    # Looping through each image path
    for image_path in image_paths:
        # Flag to determine if the image should be skipped
        skip_image = False  # Initialize the flag
        # Load the image in grayscale mode using OpenCV
        img = cv2.imread(image_path, 0)
        # Convert the grayscale image to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Crop the image using the coordinates extracted from 'coord_str'
        img = img[x_start:x_end, y_start:y_end]
        
        # Apply bilateral filter to the image for noise reduction while keeping edges sharp
        # Parameters: source image, diameter of pixel neighborhood (9), 
        # color space filter sigma (75), coordinate space filter sigma (75)
        img_DN = cv2.bilateralFilter(img, 9, 75, 75)
        gray = cv2.cvtColor(img_DN, cv2.COLOR_BGR2GRAY) # Convert the filtered image to grayscale
        
        # Apply adaptive thresholding to convert the grayscale image to a binary image
        # Parameters: source image, max value (255), adaptive method (Gaussian),
        # threshold type (binary), block size (15), constant subtracted from mean (3)        
        thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 15, 3)
        # Define a sharpening kernel
        kernel = np.array([[-1, -1, -1],
                           [-1,  9, -1],
                           [-1, -1, -1]])
        # Apply the kernel to the thresholded image to sharpen it
        sharpened = cv2.filter2D(thresh, -1, kernel)
        # Apply the Laplacian operator to highlight regions of rapid intensity change
        laplacian = cv2.Laplacian(sharpened, cv2.CV_64F)
        # Convert the Laplacian result to an 8-bit absolute value to ensure proper image format
        laplacian_abs = cv2.convertScaleAbs(laplacian)
        
        # Find contours in the laplacian_abs image using external contours only
        # and storing each contour's points as-is (without any compression)        
        contours, _ = cv2.findContours(laplacian_abs, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        
        # Define a minimum contour area to filter out small contours
        min_contour_area = 50
        valid_contours = []
        
        # Iterate through each contour found
        for contour in contours:
            # Calculate the area of the contour
            contour_area = cv2.contourArea(contour)
            # Add contour to valid_contours list if its area is above the threshold
            if contour_area >= min_contour_area:
                valid_contours.append(contour)
        # If no valid contours are found, set the skip_image flag to True
        if not valid_contours:  # Check if the list is empty
            skip_image = True
            
        # Initialize a list to store filtered contours after nested contour check
        filtered_contours = []
        
        # Iterate through valid_contours to filter out nested contours
        for i, contour in enumerate(valid_contours):
            is_nested = False
            # Get the bounding rectangle of the contour
            (x, y, w, h) = cv2.boundingRect(contour)
            # Check if this contour is nested within another
            for j, inner_contour in enumerate(valid_contours):
                if i != j:
                    (x_inner, y_inner, w_inner, h_inner) = cv2.boundingRect(inner_contour)
                    # If the inner contour is completely within the outer contour, mark as nested        
                    if x_inner >= x and y_inner >= y and x_inner + w_inner <= x + w and y_inner + h_inner <= y + h:
                        is_nested = True
                        break
            # If contour is not nested, add it to filtered_contours list
            if not is_nested:
                filtered_contours.append(contour)
        # If no filtered_contours are found and skip_image is not already set to True, set it to True
        if not filtered_contours and not skip_image:  # Check if the list is empty and skip_image is still False
            skip_image = True
            
        # Initialize a list to store contours that are identified as rhomboid shapes
        rhomboid_contours = []

        # Process each contour in filtered_contours to identify rhomboids
        for contour in filtered_contours:
            # Approximate contour to a polygon and calculate its perimeter
            epsilon = 0.05 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Check if the approximated polygon has 4 sides (potential rhomboid)
            if len(approx) == 4:
                # Calculate lengths between points in the contour
                lengths = []
                for i in range(4):
                    p1 = approx[i][0]
                    p2 = approx[(i + 1) % 4][0]
                    length = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
                    lengths.append(length)

                # Sort the lengths
                sorted_lengths = sorted(lengths)

                # Check if two sides are at least 30% longer than the other two
                if sorted_lengths[2] >= 1.3 * sorted_lengths[1] and sorted_lengths[3] >= 1.3 * sorted_lengths[0]:
                    rectangular_contours.append(contour)
        if not rectangular_contours and not skip_image:  # Check if the list is empty and skip_image is still False
            skip_image = True            
        # If any of the lists were empty, skip the current image_path
        if skip_image:
            # Save the original (cropped) image to the output path            
            output_filename = os.path.basename(image_path)
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, img)
            # Add a new row to the DataFrame with empty values for analysis results
            new_row = pd.DataFrame({
            "ImageName": [os.path.basename(image_path)],
            "RectangularArea_micrometer^2": [""],
            "Volume_micrometer^3": [""],
            "SurfaceArea_micrometer^2": [""],
            "Number_of_Moles" : [""]
            })
            # Concatenate the new row to the DataFrame
            df = pd.concat([df, new_row], ignore_index=True)
            # Continue to the next image in the loop
            continue
            
        # Increment the found_crystals counter if valid rhomboid contours are found
        found_crystals += 1
        # If more than one rhomboid contour is found, sort them by area and keep only the largest one
        if len(rectangular_contours) > 1:
            # Sort the contours by area in descending order
            rectangular_contours.sort(key=cv2.contourArea, reverse=True)
            # Keep only the largest contour
            rectangular_contours = [rectangular_contours[0]]
            
        # Make a copy of the cropped image for drawing contours
        rectangular_image = img.copy()
        # Draw green contours around the identified rhomboids
        cv2.drawContours(rhomboid_image, rectangular_contours, -1, (0, 255, 0), thickness=1)

        # Process each rhomboid contour for detailed analysis        
        for contour in rhomboid_contours:
            # Get the rotated bounding rectangle
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            rhomboid_color = (255, 0, 0)  # Blue color
            # Draw the rectangle
            cv2.drawContours(rectangular_image, [rhomboid_vertices], -1, rhomboid_color, thickness=1)
            
            # Calculate the area of the rectangle
            side1 = np.linalg.norm(box[0] - box[1])
            side2 = np.linalg.norm(box[1] - box[2])
            area = side1 * side2
            # Convert the area from pixels to micrometers
            area_micrometer = area * (conversion_rate ** 2)

        # Second part: 3D mesh volume calculation
        volume = 0.0
        # Iterate over the faces of the mesh and sum up the volumes of the tetrahedra formed with the origin
        for face in your_mesh.triangles:
            volume += tetrahedron_volume(face[0], face[1], face[2], np.array([0., 0., 0.]))
        
        # Calculate scaling factors for the mesh based on the rhomboid dimensions
        scale_x, scale_z = diagonal1 * conversion_rate, diagonal2 * conversion_rate
        # This is based on the relationship between the 210 and the 001 faces.
        # This number is not necessary correct for the Orthorhombic Bipyramidal crystals and therefore must be checked first
        scale_y = scale_x/4 
        # Limit the scale_y to a maximum value
        if scale_y > 10:
            scale_y = 10
        
        # Obtain the original dimensions of the mesh
        original_dims = your_mesh.bounds[1] - your_mesh.bounds[0]
        original_x, original_y, original_z = original_dims

        # Calculate the new scaling factors for each dimension
        new_scale_x = scale_x / original_x
        new_scale_y = scale_y / original_y
        new_scale_z = scale_z / original_z

        # Apply the new scaling factors to the mesh
        scaled_mesh = your_mesh.apply_scale([new_scale_x, new_scale_y, new_scale_z])

        # Recalculate the volume of the scaled mesh
        scaled_volume = 0.0 # initialize the volume size parameter
        for face in scaled_mesh.triangles:
            scaled_volume += tetrahedron_volume(face[0], face[1], face[2], np.array([0., 0., 0.]))

        # Calculate surface area of the scaled mesh
        surface_area = np.sum(scaled_mesh.area_faces)
        # Calculate surface area in m^2
        surface_area_m2 = surface_area * 1e-12  # Convert from um^2 to m^2
        # Convert volume from micrometer^3 to cm^3
        scaled_volume_cm3 = scaled_volume / 1e12  # 1 cm^3 = 10^12 micrometer^3
        # Calculate the number of moles
        number_of_moles = scaled_volume_cm3 / molar_volume_cm3
        
        # Add the calculated data to the DataFrame
        new_row = pd.DataFrame({
            "ImageName": [os.path.basename(image_path)],
            "RectangularArea_micrometer^2": [area_micrometer],
            "Volume_micrometer^3": [scaled_volume],
            "SurfaceArea_micrometer^2": [surface_area],
            "Number_of_Moles": [number_of_moles]
        })

        df = pd.concat([df, new_row], ignore_index=True)
        
        # Save the processed image with drawn contours
        output_filename = os.path.basename(image_path)
        output_path = os.path.join(output_folder, output_filename)
        cv2.imwrite(output_path, rhomboid_image)

        # add diagonal length to crystal_size database to calculate the crystal size
        new_row = pd.DataFrame({
            'exp_number': [exp_number],
            'folder_number': folder_number,
            'diagonal_1': [diagonal1 * conversion_rate],  # Convert to micrometers
        })
        crystal_size_df = pd.concat([crystal_size_df, new_row], ignore_index=True)

    # Save the DataFrame as a CSV file
    output_csv_path = os.path.join(output_folder, output_csv_filename)
    df.to_csv(output_csv_path, index=False)
    output_csv_path = os.path.join(output_folder, output_csv_filename)
    # Read the CSV file that was just saved
    csv_df = pd.read_csv(output_csv_path)
    # Read an additional Excel file for comparison
    excel_df = pd.read_excel(f'D:/Radium/Results/time_{exp_number}.xlsx')
    # Print the number of rows in both DataFrames for comparison
    print(csv_df.shape[0])
    print(excel_df.shape[0])
    
    # Check if the number of rows in both DataFrames is the same
    if csv_df.shape[0] != excel_df.shape[0]:
        print("Warning: The number of columns in the CSV and Excel files are not the same.")
    # Store the number of rows in each DataFrame
    csv_rows = csv_df.shape[0]
    excel_rows = excel_df.shape[0]
    if csv_rows != excel_rows:
        print("Warning: The number of rows in the CSV and Excel files are not the same.")
        print(csv_df.shape[0])
        print(excel_df.shape[0])
        # Trim the Excel DataFrame if it has more rows than the CSV DataFrame
        if excel_rows > csv_rows:
            excel_df = excel_df.iloc[:csv_rows]
    # Join the columns from the Excel DataFrame to the CSV DataFrame
    joined_df = pd.concat([csv_df, excel_df], axis=1)

    # Save the joined DataFrame to a new CSV file
    joined_df.to_csv(output_csv_path, index=False)
    
    # Print summary information about the processing
    print(found_crystals, exp_number, folder_name)
    print("-----------------")
    
    # Add 'exp_number' and 'folder_name' to the DataFrame
    df['exp_number'] = exp_number
    df['folder_name'] = folder_name
    df['time_seconds'] = excel_df['time_min']*60

    # Concatenate this DataFrame with the main DataFrame
    main_df = pd.concat([main_df, df], ignore_index=True)
    

In [ ]:
# Save the main DataFrame to a single CSV file
main_df.to_csv("D:/Radium/Results_auto/OB_results/All_Experiments_OB.csv", index=False)